# 📘 代理架构 12：图推理

欢迎来到我们系列的第十二个notebook。今天，我们探索**图推理**架构，这是一种利用图结构来表示和推理复杂关系的模式。

图推理允许代理理解实体之间的连接，执行多跳推理，并回答需要遍历知识图的复杂问题。

### 定义
一个**图 / World-Model Memory** 是代理架构 其中知识 存储在一个structured graph database. 信息 被重新presented 作为nodes (entities like people, places, concepts)和edges (the relationships between them). 这creates 一个dynamic "world 模型" 那个agent 可以reason over.

### 高级工作流程

1. **信息 Ingestion:** 代理 receives unstructured或semi-structured数据(text, documents, API responses).
2. **Knowledge Extraction:** 一个LLM-powered过程parses information, identifying key entities和relationships 那个connect them.
3. **图 Update:** extracted nodes和edges 是added来or updated在一个persistent graph database (like Neo4j).
4. **Question 答案ing / Reasoning:** 当 asked 问题, 代理:
 a. Converts natural language问题into 一个formal graph查询language (e.g., Cypher为Neo4j).
 b. Executes查询against graph来retrieve relevant subgraphs或facts.
 c. Synthesizes query结果into 一个natural language answer.

### 何时使用/应用场景
* **Enterprise Knowledge Assistants:** 构建 一个queryable 模型的company's projects, employees,和customers从internal documents.
* **Advanced Research Assistants:** 创建 一个dynamic knowledge base的scientific field通过ingesting research papers.
* **Complex System Diagnostics:** Modeling 系统's components和their dependencies来diagnose failures.

### 优点和缺点
* **优点：**
 * **Structured & Explainable:** knowledge 是highly organized. An答案can 是explained通过showing exact path在graph 那个led来it.
 * **Enables Complex Reasoning:** Excels在nswering "multi-hop"问题那个require connecting disparate pieces的information through relationships.
* **缺点：**
 * **Upfront Complexity:** Requires 一个定义良好的 schema和一个robust extraction 过程的核心。
 * **Keeping 图 Updated:** 可以be challenging来manage updates, resolve conflicting information,和prune outdated facts over time (knowledge lifecycle management).

## 阶段0：基础与设置

我们将安装 libraries, including Neo4j driver,和配置我们的environment. **Crucially, 你必须have 一个running Neo4j instance和一个`.env` file与its credentials.**

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain_community neo4j

In [2]:
import os from typing import List, Dict, Any, Optional from dotenv import load_dotenv

# Pydanticfordata modeling from pydantic import BaseModel, Field

# LangChain components from langchain_nebius import ChatNebius from langchain_community.graphs import Neo4jGraph from langchain.chains import GraphCypherQAChain from langchain_core.prompts import ChatPromptTemplate from langchain_core.pydantic_v1 import BaseModel as V1BaseModel

# 用于美观打印 from rich.console import Console from rich.markdown import Markdown

# --- API KeyandTracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Graph Memory (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD"] for var in required_vars:
 if var not in os.environ:
 print(f"警告: Environment variable {var} not set.")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1：构建图构建代理

这是摄取管道的核心。我们需要一个可以读取非结构化文本并以结构化格式提取实体和关系的代理。我们将使用具有结构化输出能力（Pydantic）的LLM作为我们的知识提取器。

In [3]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)

# Connect我们的Neo4j database
try:
 graph = Neo4jGraph()
 # Clear graphfor一个clean run
.query("MATCH (n) DETACH DELETE n")
except Exception as e:
 console.print(f"[bold red]连接Neo4j失败：{e}。请检查您的凭据and连接。[/bold red]")
 graph = None

# Pydanticmodelforstructured extraction (using LangChain's v1 BaseModelforcompatibilitywitholder structuredoutputmethods)
class Node(V1BaseModel):
 id: str = Field(description="Unique nameoridentifierforentity.")
 type: str = Field(description="typeorlabel的entity (e.g., Person, Company, Product).")

class Relationship(V1BaseModel):
 source: Node
 target: Node
 type: str = Field(description="关系类型 (e.g., WORKS_FOR, ACQUIRED).")

class KnowledgeGraph(V1BaseModel):
 """A graph的nodesandrelationships."""
 relationships: List[Relationship]

# Graph Maker Agent
def get_graph_maker_chain():
 extractor_llm = llm.with_structured_output(KnowledgeGraph)
 prompt = ChatPromptTemplate.from_messages([
 ("system", "你is提取信息的专家 fromtext中构建知识。 提取所有实体（Node）and关系 from提供的text中. 关系class型应该is全大写的动词, like 'WORKS_FOR'or'ACQUIRED'."),
 ("human", "Extract 一个knowledge graphfromfollowing text:\n\n{text}")
 ])
 returnprompt | extractor_llm

graph_maker_agent = get_graph_maker_chain()
print("成功fully connected来Neo4janddefined Graph Maker Agent.")

成功fully connected来Neo4janddefined 图 Maker Agent.


## 阶段2： Ingesting Knowledge和构建 World Model

Now, 我们将 feed我们的代理 一个series的related但separate documents. 代理将过程each one和progressively 构建up我们的corporate knowledge 图的状态。这simulates how 一个real 系统 would学习over time 作为new信息becomes available.

In [4]:
unstructured_documents = [
 "On May 15, 2023, global tech giant AlphaCorp announced its acquisition的startup Beta解决方案s, 一个leaderincloud-native database technology.",
 "Dr. Evelyn Reed, 一个renowned AI researcher, has been Chief Science OfficerinAlphaCorp since 2021. She leads team responsibleforQuantumLeap AI platform.",
 "Innovate Inc.'s flagship product, NeuraGen, is seen as 一个direct competitor来AlphaCorp's QuantumLeap AI. Meanwhile, Innovate Inc. recently hired Johnathan Miles as their新的CTO."
] fori, docinenumerate(unstructured_documents):
 console.print(f"--- 摄取文档 {i+1} ---")
 try:
 kg_data = graph_maker_agent.invoke({"text": doc})
 if kg_data.relationships:
.add_graph_documents(graph_documents=kg_data.relationships, include_source=False)
 console.print(f"[green]成功fully added {len(kg_data.relationships)} relationships来.[/green]")
 else:
 console.print("[yellow]No relationships extracted.[/yellow]")
 except Exception as e:
 console.print(f"[red]Failed来process document: {e}[/red]")

console.print("--- ✅ Knowledge Graph Ingestion Complete ---")
console.print("\n--- Graph Schema ---")
console.print(.schema)

--- 摄取文档 1 ---
成功fully added 1 relationships来图的状态。
--- 摄取文档 2 ---
成功fully added 2 relationships来图的状态。
--- 摄取文档 3 ---
成功fully added 2 relationships来图的状态。
--- ✅ Knowledge 图 Ingestion Complete ---



--- 图 Schema ---


节点 properties: [{'properties': [('id', 'STRING')], 'labels': ['Product']}, {'properties': [('id', 'STRING')], 'labels': ['Person']}, {'properties': [('id', 'STRING')], 'labels': ['Company']}]
Relationship properties: []
Relationships: [(:Company)-[:PRODUCES]->(:Product), (:Person)-[:WORKS_FOR]->(:Company), (:Product)-[:COMPETES_WITH]->(:Product), (:Company)-[:ACQUIRED]->(:Company)]


## 阶段3： 构建 图-Querying Agent 与our knowledge graph populated，我们need 代理 那个can 使用it. 这包括 一个**Text-to-Cypher** 过程的核心。 代理将接收一个user's natural language question, convert它into 一个Cypher查询using graph schema 作为context,执行query,和then synthesize结果into 一个human-readable answer.

In [5]:
# LangChain has 一个built-in chainforthis，但我们将 inspect components
#来understand how it works.
cypher_generation_prompt = ChatPromptTemplate.from_template(
 """你is一位专业的Neo4j Cypher查询开发者. 你的任务is将用户的自然语言问题转换for有效的Cypher查询.
你必须使用提供的Graph模式 来构建查询. not要使用任何Node labelsorrelationship types 那个are notinschema.
Return ONLY Cypher query,withno additional textorexplanations.

Graph Schema:
{schema}

User 问题：
{question}
"""
)

cypher_response_prompt = ChatPromptTemplate.from_template(
 """你is一个assistant 那个提供 clear, natural language答案basedinquery resultsfrom一个knowledge。
Use contextfromgraph查询result回答 user's original question.

User 问题： {question}
Query result： {context}
"""
)

def query_graph(question: str) -> Dict[str, Any]:
 """full Text-to-Cypherandsynthesis pipeline."""
 console.print(f"\n[bold]问题：[/bold] {question}")
 
 # 1. Generate Cypher Query
 console.print("--- ➡️ 生成Cypher查询 ---")
 cypher_chain = cypher_generation_prompt | llm
 generated_cypher = cypher_chain.invoke({"schema":。schema, "question": question}).content
 console.print(f"[cyan]Generated Cypher:\n{generated_cypher}[/cyan]")
 
 # 2. Execute Cypher Query
 console.print("--- ⚡ 执行查询 ---")
 try:
 context =.query(generated_cypher)
 console.print(f"[yellow]Query result：\n{context}[/yellow]")
 except Exception as e:
 console.print(f"[red]Cypher Query Failed: {e}[/red]")
 return{"answer": "I was unable来execute 一个query来find answer来your question."}
 
 # 3. Synthesize 最终答案
 console.print("--- 🗣️ Synthesizing 最终答案 ---")
 synthesis_chain = cypher_response_prompt | llm
 答案= synthesis_chain.invoke({"question": question, "context": context}).content
 
 return{"answer": answer}

print("Graph-Querying Agent defined successfully.")

图-Querying Agent 定义成功.


## 阶段4： 演示与分析

现在进行最终测试。我们将向代理提出从简单事实检索到需要连接所有三个源文档信息的复杂多跳推理的问题。

In [6]:
# 测试1： Simple fact retrieval (requires infofromdoc 2)
result1 = query_graph("Who worksforAlphaCorp?")
console.print("\n--- 最终答案 ---")
console.print(Markdown(result1['answer']))

# 测试2： Another 简单的 fact retrieval (requires infofromdoc 1)
result2 = query_graph("What company did AlphaCorp acquire?")
console.print("\n--- 最终答案 ---")
console.print(Markdown(result2['answer']))

# 测试3： multi-hop reasoning问题(requires infofromall 3 docs)
result3 = query_graph("What companies competewithproducts madepasscompany 那个acquired Beta解决方案s?")
console.print("\n--- 最终答案 ---")
console.print(Markdown(result3['answer']))


问题： Who worksforAlphaCorp?
--- ➡️ 生成 Cypher Query ---
Generated Cypher:
MATCH (p:Person)-[:WORKS_FOR]->(c:Company {id: 'AlphaCorp'}) RETURN p.id
--- ⚡ 执行查询 ---
Query 结果：
[{'p.id': 'Dr. Evelyn Reed'}]
--- 🗣️ Synthesizing 最终答案 ---



--- 最终答案 ---


Dr. Evelyn Reed works 为AlphaCorp.


问题： What company did AlphaCorp acquire?
--- ➡️ 生成 Cypher Query ---
Generated Cypher:
MATCH (:Company {id: 'AlphaCorp'})-[:ACQUIRED]->(acquired_company:Company)
RETURN acquired_company.id
--- ⚡ 执行查询 ---
Query 结果：
[{'acquired_company.id': 'Beta解决方案s'}]
--- 🗣️ Synthesizing 最终答案 ---



--- 最终答案 ---


AlphaCorp acquired BetaSolutions.


问题： What companies competewithproducts madepasscompany 那个acquired Beta解决方案s?
--- ➡️ 生成 Cypher Query ---
Generated Cypher:
MATCH (acquirer:Company)-[:ACQUIRED]->(:Company {id: 'Beta解决方案s'})
MATCH (acquirer)-[:PRODUCES]->(product:Product)
MATCH (product)-[:COMPETES_WITH]->(competitor_product:Product)
MATCH (competitor_company:Company)-[:PRODUCES]->(competitor_product)
RETURN DISTINCT competitor_company.id
--- ⚡ 执行查询 ---
Query 结果：
[{'competitor_company.id': 'Innovate Inc.'}]
--- 🗣️ Synthesizing 最终答案 ---



--- 最终答案 ---


Innovate Inc. competes 与the products made 通过the company 那个acquired BetaSolutions.

### 结果分析

demonstration highlights profound advantage的graph-based world 模型:

- first two问题were 简单的 lookups. 代理 successfully converted问题into Cypher, queried graph,和found direct relationships.

- third问题is crucial one. 一个standard RAG 代理 会fail here. It might找到document 关于acquisition和document 关于competitor，但it 会struggle来connect them. It lacks explicit relational structure来understand 那个"AlphaCorp"在document 1 是same entity 作为"AlphaCorp"在documents 2和3.

- Our graph-based 代理, however, solved it与ease. 我们can trace它的logic directly从generated Cypher query:
 1. `MATCH (acquirer:Company)-[:ACQUIRED]->(:Company {id: 'Beta解决方案s'})`: First,找到company 那个acquired Beta解决方案s (结果： AlphaCorp).
 2. `MATCH (acquirer)-[:PRODUCES]->(product:Product)`: Next,找到products produced通过那个company (结果： QuantumLeap AI).
 3. `MATCH (product)-[:COMPETES_WITH]->(competitor_product:Product)`: Then,找到products 那个compete与那个product (结果： NeuraGen).
 4. `MATCH (competitor_company:Company)-[:PRODUCES]->(competitor_product)`: 最终ly,找到company 那个produces competing product (结果： Innovate Inc.).

这ability来traverse relationships和synthesize information从different sources 是superpower的图 / World-Model 架构.答案is不just retrieved;它被重新asoned.

## 总结

在这个notebook中，我们have constructed 一个完成 代理ic 系统 built around 一个**图 / World-Model Memory**. 我们demonstrated full lifecycle: ingesting unstructured data, using 一个LLM来构建一个structured knowledge graph,和then using 那个graph回答 复杂的, multi-hop问题那个require genuine reasoning.

这种架构 represents 一个significant leap在capability over 简单的r memory 系统。 通过creating 一个explicit, queryable 模型的world，我们give我们的代理ability来connect disparate facts和uncover hidden insights. 虽然challenges的maintaining 这个graph over time 被重新al, potential为building deeply knowledgeable和explainable AI assistants makes 这个one最exciting和强大的 模式s在modern 代理ic design.